## Hypothesis
My physical activity increases as my education life intensifies.

In [1]:
import xml.etree.ElementTree as et
import pandas as pd
import scipy.stats as stat
from matplotlib import pyplot as plt
import seaborn as sns

In [48]:
with open("apple_health_export/export.xml", "r") as f:
    my_data = et.parse(f)

print(my_data.getroot()[6].attrib)

my_count = my_data.getroot()[6].attrib
print(my_count["type"], my_count["creationDate"], my_count["value"])

{'type': 'HKQuantityTypeIdentifierStepCount', 'sourceName': 'Onur’un iPhone’u', 'sourceVersion': '13.5.1', 'device': '<<HKDevice: 0x2805f4eb0>, name:iPhone, manufacturer:Apple Inc., model:iPhone, hardware:iPhone12,1, software:13.5.1>', 'unit': 'count', 'creationDate': '2020-08-29 11:33:43 +0300', 'startDate': '2020-08-29 11:23:16 +0300', 'endDate': '2020-08-29 11:32:13 +0300', 'value': '55'}
HKQuantityTypeIdentifierStepCount 2020-08-29 11:33:43 +0300 55


In [49]:
my_dict = my_data.getroot()
print(len(my_dict))

118441


In [50]:
record_list = [x.attrib for x in my_dict.iter('Record') if (x.attrib["type"] != "HKCategoryTypeIdentifierHeadphoneAudioExposureEvent") and (x.attrib["type"] != "HKQuantityTypeIdentifierBodyMass") and (x.attrib["type"] != "HKQuantityTypeIdentifierHeight")]
record_list

[{'type': 'HKQuantityTypeIdentifierStepCount',
  'sourceName': 'Onur’un iPhone’u',
  'sourceVersion': '13.5.1',
  'device': '<<HKDevice: 0x2805f4eb0>, name:iPhone, manufacturer:Apple Inc., model:iPhone, hardware:iPhone12,1, software:13.5.1>',
  'unit': 'count',
  'creationDate': '2020-08-29 11:33:43 +0300',
  'startDate': '2020-08-29 11:23:16 +0300',
  'endDate': '2020-08-29 11:32:13 +0300',
  'value': '55'},
 {'type': 'HKQuantityTypeIdentifierStepCount',
  'sourceName': 'Onur’un iPhone’u',
  'sourceVersion': '13.5.1',
  'device': '<<HKDevice: 0x2805f4eb0>, name:iPhone, manufacturer:Apple Inc., model:iPhone, hardware:iPhone12,1, software:13.5.1>',
  'unit': 'count',
  'creationDate': '2020-08-29 11:44:33 +0300',
  'startDate': '2020-08-29 11:32:15 +0300',
  'endDate': '2020-08-29 11:42:16 +0300',
  'value': '667'},
 {'type': 'HKQuantityTypeIdentifierStepCount',
  'sourceName': 'Onur’un iPhone’u',
  'sourceVersion': '13.5.1',
  'device': '<<HKDevice: 0x2805f4eb0>, name:iPhone, manufactu

In [51]:
for record_dict in record_list:
    record_dict["value"] = float(record_dict["value"])

record_list

[{'type': 'HKQuantityTypeIdentifierStepCount',
  'sourceName': 'Onur’un iPhone’u',
  'sourceVersion': '13.5.1',
  'device': '<<HKDevice: 0x2805f4eb0>, name:iPhone, manufacturer:Apple Inc., model:iPhone, hardware:iPhone12,1, software:13.5.1>',
  'unit': 'count',
  'creationDate': '2020-08-29 11:33:43 +0300',
  'startDate': '2020-08-29 11:23:16 +0300',
  'endDate': '2020-08-29 11:32:13 +0300',
  'value': 55.0},
 {'type': 'HKQuantityTypeIdentifierStepCount',
  'sourceName': 'Onur’un iPhone’u',
  'sourceVersion': '13.5.1',
  'device': '<<HKDevice: 0x2805f4eb0>, name:iPhone, manufacturer:Apple Inc., model:iPhone, hardware:iPhone12,1, software:13.5.1>',
  'unit': 'count',
  'creationDate': '2020-08-29 11:44:33 +0300',
  'startDate': '2020-08-29 11:32:15 +0300',
  'endDate': '2020-08-29 11:42:16 +0300',
  'value': 667.0},
 {'type': 'HKQuantityTypeIdentifierStepCount',
  'sourceName': 'Onur’un iPhone’u',
  'sourceVersion': '13.5.1',
  'device': '<<HKDevice: 0x2805f4eb0>, name:iPhone, manufactu

## Identifier Types

Not to be considered:

* HKCategoryTypeIdentifierHeadphoneAudioExposureEvent
    * to be removed at the XML parsing - it does not contain any numeric value
* HKQuantityTypeIdentifierBodyMass
* HKQuantityTypeIdentifierHeight

To be considered:

* HKQuantityTypeIdentifierHeadphoneAudioExposure

* HKQuantityTypeIdentifierActiveEnergyBurned
* HKQuantityTypeIdentifierBasalEnergyBurned



* HKQuantityTypeIdentifierAppleWalkingSteadiness
* HKQuantityTypeIdentifierDistanceWalkingRunning
* HKQuantityTypeIdentifierFlightsClimbed
* HKQuantityTypeIdentifierStepCount
* HKQuantityTypeIdentifierWalkingAsymmetryPercentage
* HKQuantityTypeIdentifierWalkingDoubleSupportPercentage
* HKQuantityTypeIdentifierWalkingSpeed
* HKQuantityTypeIdentifierWalkingStepLength

Dataframe consisting of all the data

In [52]:
my_all_data = pd.DataFrame(record_list)

# Conversion of dates and times into datetime objects
my_all_data["creationDate"] = pd.to_datetime(my_all_data["creationDate"])
my_all_data["startDate"] = pd.to_datetime(my_all_data["startDate"])
my_all_data["endDate"] = pd.to_datetime(my_all_data["endDate"])

# Source name is "Onur'un iPhone'u" in all cases
my_all_data = my_all_data.drop(columns = ["sourceName"])

my_all_data

,type,sourceVersion,device,unit,creationDate,startDate,endDate,value
0,HKQuantityTypeIdentifierStepCount,13.5.1,"<<HKDevice: 0x2805f4eb0>, name:iPhone, manufac...",count,2020-08-29 11:33:43+03:00,2020-08-29 11:23:16+03:00,2020-08-29 11:32:13+03:00,55.0
1,HKQuantityTypeIdentifierStepCount,13.5.1,"<<HKDevice: 0x2805f4eb0>, name:iPhone, manufac...",count,2020-08-29 11:44:33+03:00,2020-08-29 11:32:15+03:00,2020-08-29 11:42:16+03:00,667.0
2,HKQuantityTypeIdentifierStepCount,13.5.1,"<<HKDevice: 0x2805f4eb0>, name:iPhone, manufac...",count,2020-08-29 11:54:40+03:00,2020-08-29 11:42:16+03:00,2020-08-29 11:48:59+03:00,501.0
3,HKQuantityTypeIdentifierStepCount,11.0,"<<HKDevice: 0x2805f5270>, name:iPhone, manufac...",count,2017-11-23 22:32:12+03:00,2017-11-23 22:20:45+03:00,2017-11-23 22:22:34+03:00,22.0
4,HKQuantityTypeIdentifierStepCount,11.0,"<<HKDevice: 0x2805f5270>, name:iPhone, manufac...",count,2017-11-24 08:21:36+03:00,2017-11-24 07:45:13+03:00,2017-11-24 07:55:11+03:00,122.0
...,...,...,...,...,...,...,...,...
117978,HKQuantityTypeIdentifierAppleWalkingSteadiness,2886.1.7,"<<HKDevice: 0x280540500>, name:iPhone, manufac...",%,2023-10-12 09:04:49+03:00,2023-10-05 03:00:00+03:00,2023-10-12 03:00:00+03:00,1.0
117979,HKQuantityTypeIdentifierAppleWalkingSteadiness,2886.1.7,"<<HKDevice: 0x280540500>, name:iPhone, manufac...",%,2023-10-19 08:12:23+03:00,2023-10-12 03:00:00+03:00,2023-10-19 03:00:00+03:00,1.0
117980,HKQuantityTypeIdentifierAppleWalkingSteadiness,2886.1.7,"<<HKDevice: 0x280540500>, name:iPhone, manufac...",%,2023-10-26 08:39:04+03:00,2023-10-19 03:00:00+03:00,2023-10-26 03:00:00+03:00,1.0
117981,HKQuantityTypeIdentifierAppleWalkingSteadiness,2890.0.8.0.5,"<<HKDevice: 0x280540550>, name:iPhone, manufac...",%,2023-11-02 09:05:53+03:00,2023-10-26 03:00:00+03:00,2023-11-02 03:00:00+03:00,1.0


Dataframes related to physical activity

In [53]:
my_steps = my_all_data[my_all_data["type"] == "HKQuantityTypeIdentifierStepCount"].reset_index().drop(columns = ["index"])
my_steps.head()

,type,sourceVersion,device,unit,creationDate,startDate,endDate,value
0,HKQuantityTypeIdentifierStepCount,13.5.1,"<<HKDevice: 0x2805f4eb0>, name:iPhone, manufac...",count,2020-08-29 11:33:43+03:00,2020-08-29 11:23:16+03:00,2020-08-29 11:32:13+03:00,55.0
1,HKQuantityTypeIdentifierStepCount,13.5.1,"<<HKDevice: 0x2805f4eb0>, name:iPhone, manufac...",count,2020-08-29 11:44:33+03:00,2020-08-29 11:32:15+03:00,2020-08-29 11:42:16+03:00,667.0
2,HKQuantityTypeIdentifierStepCount,13.5.1,"<<HKDevice: 0x2805f4eb0>, name:iPhone, manufac...",count,2020-08-29 11:54:40+03:00,2020-08-29 11:42:16+03:00,2020-08-29 11:48:59+03:00,501.0
3,HKQuantityTypeIdentifierStepCount,11.0,"<<HKDevice: 0x2805f5270>, name:iPhone, manufac...",count,2017-11-23 22:32:12+03:00,2017-11-23 22:20:45+03:00,2017-11-23 22:22:34+03:00,22.0
4,HKQuantityTypeIdentifierStepCount,11.0,"<<HKDevice: 0x2805f5270>, name:iPhone, manufac...",count,2017-11-24 08:21:36+03:00,2017-11-24 07:45:13+03:00,2017-11-24 07:55:11+03:00,122.0


In [54]:
my_distances = my_all_data[my_all_data["type"] == "HKQuantityTypeIdentifierDistanceWalkingRunning"].reset_index().drop(columns = ["index"])
my_distances.head()

,type,sourceVersion,device,unit,creationDate,startDate,endDate,value
0,HKQuantityTypeIdentifierDistanceWalkingRunning,13.5.1,"<<HKDevice: 0x2806c1090>, name:iPhone, manufac...",km,2020-08-29 11:33:43+03:00,2020-08-29 11:23:16+03:00,2020-08-29 11:32:13+03:00,0.03644
1,HKQuantityTypeIdentifierDistanceWalkingRunning,13.5.1,"<<HKDevice: 0x2806c1090>, name:iPhone, manufac...",km,2020-08-29 11:44:33+03:00,2020-08-29 11:32:15+03:00,2020-08-29 11:42:16+03:00,0.47539
2,HKQuantityTypeIdentifierDistanceWalkingRunning,13.5.1,"<<HKDevice: 0x2806c1090>, name:iPhone, manufac...",km,2020-08-29 11:54:40+03:00,2020-08-29 11:42:16+03:00,2020-08-29 11:48:59+03:00,0.35122
3,HKQuantityTypeIdentifierDistanceWalkingRunning,11.0,"<<HKDevice: 0x2806cc1e0>, name:iPhone, manufac...",km,2017-11-23 22:32:12+03:00,2017-11-23 22:20:45+03:00,2017-11-23 22:22:34+03:00,0.00980
4,HKQuantityTypeIdentifierDistanceWalkingRunning,11.0,"<<HKDevice: 0x2806cc1e0>, name:iPhone, manufac...",km,2017-11-24 08:21:36+03:00,2017-11-24 07:45:13+03:00,2017-11-24 07:55:11+03:00,0.07920


In [55]:
my_climbed = my_all_data[my_all_data["type"] == "HKQuantityTypeIdentifierFlightsClimbed"].reset_index().drop(columns = ["index"])
my_climbed.head()

,type,sourceVersion,device,unit,creationDate,startDate,endDate,value
0,HKQuantityTypeIdentifierFlightsClimbed,13.5.1,"<<HKDevice: 0x2806663f0>, name:iPhone, manufac...",count,2020-08-29 12:01:30+03:00,2020-08-29 11:48:05+03:00,2020-08-29 11:48:51+03:00,4.0
1,HKQuantityTypeIdentifierFlightsClimbed,11.0,"<<HKDevice: 0x280666cb0>, name:iPhone, manufac...",count,2017-11-24 08:21:37+03:00,2017-11-24 08:08:41+03:00,2017-11-24 08:08:41+03:00,1.0
2,HKQuantityTypeIdentifierFlightsClimbed,11.0,"<<HKDevice: 0x280666cb0>, name:iPhone, manufac...",count,2017-11-24 16:37:17+03:00,2017-11-24 10:54:49+03:00,2017-11-24 10:54:49+03:00,1.0
3,HKQuantityTypeIdentifierFlightsClimbed,11.0,"<<HKDevice: 0x280666cb0>, name:iPhone, manufac...",count,2017-11-24 16:37:17+03:00,2017-11-24 12:31:57+03:00,2017-11-24 12:31:57+03:00,1.0
4,HKQuantityTypeIdentifierFlightsClimbed,11.0,"<<HKDevice: 0x280666cb0>, name:iPhone, manufac...",count,2017-11-24 16:37:17+03:00,2017-11-24 14:06:04+03:00,2017-11-24 14:06:04+03:00,1.0


In [56]:
my_step_length = my_all_data[my_all_data["type"] == "HKQuantityTypeIdentifierWalkingStepLength"].reset_index().drop(columns = ["index"])
my_step_length.head()

,type,sourceVersion,device,unit,creationDate,startDate,endDate,value
0,HKQuantityTypeIdentifierWalkingStepLength,14.0,"<<HKDevice: 0x2805402d0>, name:iPhone, manufac...",cm,2020-09-21 09:06:06+03:00,2020-09-21 08:55:43+03:00,2020-09-21 08:56:55+03:00,84.0
1,HKQuantityTypeIdentifierWalkingStepLength,14.0,"<<HKDevice: 0x2805402d0>, name:iPhone, manufac...",cm,2020-09-21 10:32:19+03:00,2020-09-21 10:20:34+03:00,2020-09-21 10:20:50+03:00,82.0
2,HKQuantityTypeIdentifierWalkingStepLength,14.0,"<<HKDevice: 0x2805402d0>, name:iPhone, manufac...",cm,2020-09-21 10:32:19+03:00,2020-09-21 10:22:17+03:00,2020-09-21 10:22:26+03:00,90.0
3,HKQuantityTypeIdentifierWalkingStepLength,14.0,"<<HKDevice: 0x2805402d0>, name:iPhone, manufac...",cm,2020-09-21 13:01:57+03:00,2020-09-21 12:50:43+03:00,2020-09-21 12:50:47+03:00,72.0
4,HKQuantityTypeIdentifierWalkingStepLength,14.0,"<<HKDevice: 0x2805402d0>, name:iPhone, manufac...",cm,2020-09-21 13:01:57+03:00,2020-09-21 12:51:06+03:00,2020-09-21 12:52:19+03:00,74.0


In [57]:
my_walking_speed = my_all_data[my_all_data["type"] == "HKQuantityTypeIdentifierWalkingSpeed"].reset_index().drop(columns = ["index"])
my_walking_speed.head()

,type,sourceVersion,device,unit,creationDate,startDate,endDate,value
0,HKQuantityTypeIdentifierWalkingSpeed,14.0,"<<HKDevice: 0x280540190>, name:iPhone, manufac...",km/hr,2020-09-21 09:06:06+03:00,2020-09-21 08:55:43+03:00,2020-09-21 08:56:55+03:00,6.336
1,HKQuantityTypeIdentifierWalkingSpeed,14.0,"<<HKDevice: 0x280540190>, name:iPhone, manufac...",km/hr,2020-09-21 10:32:19+03:00,2020-09-21 10:20:34+03:00,2020-09-21 10:20:50+03:00,6.120
2,HKQuantityTypeIdentifierWalkingSpeed,14.0,"<<HKDevice: 0x280540190>, name:iPhone, manufac...",km/hr,2020-09-21 10:32:19+03:00,2020-09-21 10:22:17+03:00,2020-09-21 10:22:26+03:00,6.876
3,HKQuantityTypeIdentifierWalkingSpeed,14.0,"<<HKDevice: 0x280540190>, name:iPhone, manufac...",km/hr,2020-09-21 13:01:57+03:00,2020-09-21 12:50:43+03:00,2020-09-21 12:50:47+03:00,4.176
4,HKQuantityTypeIdentifierWalkingSpeed,14.0,"<<HKDevice: 0x280540190>, name:iPhone, manufac...",km/hr,2020-09-21 13:01:57+03:00,2020-09-21 12:51:06+03:00,2020-09-21 12:52:19+03:00,4.824


In [58]:
my_walking_asymmetry = my_all_data[my_all_data["type"] == "HKQuantityTypeIdentifierWalkingAsymmetryPercentage"].reset_index().drop(columns = ["index"])
my_walking_asymmetry.head()

,type,sourceVersion,device,unit,creationDate,startDate,endDate,value
0,HKQuantityTypeIdentifierWalkingAsymmetryPercen...,14.0,"<<HKDevice: 0x280540190>, name:iPhone, manufac...",%,2020-09-21 09:06:06+03:00,2020-09-21 08:55:43+03:00,2020-09-21 08:56:55+03:00,0.00
1,HKQuantityTypeIdentifierWalkingAsymmetryPercen...,14.0,"<<HKDevice: 0x280540190>, name:iPhone, manufac...",%,2020-09-21 10:32:19+03:00,2020-09-21 10:20:34+03:00,2020-09-21 10:20:50+03:00,0.07
2,HKQuantityTypeIdentifierWalkingAsymmetryPercen...,14.0,"<<HKDevice: 0x280540190>, name:iPhone, manufac...",%,2020-09-21 13:01:57+03:00,2020-09-21 12:51:06+03:00,2020-09-21 12:52:19+03:00,0.00
3,HKQuantityTypeIdentifierWalkingAsymmetryPercen...,14.0,"<<HKDevice: 0x280540190>, name:iPhone, manufac...",%,2020-09-22 12:17:56+03:00,2020-09-22 09:49:28+03:00,2020-09-22 09:50:48+03:00,0.00
4,HKQuantityTypeIdentifierWalkingAsymmetryPercen...,14.0,"<<HKDevice: 0x280540190>, name:iPhone, manufac...",%,2020-09-22 12:40:06+03:00,2020-09-22 12:18:35+03:00,2020-09-22 12:18:59+03:00,0.00


In [59]:
my_walking_double_support = my_all_data[my_all_data["type"] == "HKQuantityTypeIdentifierWalkingDoubleSupportPercentage"].reset_index().drop(columns = ["index"])
my_walking_double_support.head()

,type,sourceVersion,device,unit,creationDate,startDate,endDate,value
0,HKQuantityTypeIdentifierWalkingDoubleSupportPe...,14.0,"<<HKDevice: 0x2805e6cb0>, name:iPhone, manufac...",%,2020-09-21 09:06:06+03:00,2020-09-21 08:55:43+03:00,2020-09-21 08:56:55+03:00,0.228
1,HKQuantityTypeIdentifierWalkingDoubleSupportPe...,14.0,"<<HKDevice: 0x2805e6cb0>, name:iPhone, manufac...",%,2020-09-21 10:32:19+03:00,2020-09-21 10:20:34+03:00,2020-09-21 10:20:50+03:00,0.236
2,HKQuantityTypeIdentifierWalkingDoubleSupportPe...,14.0,"<<HKDevice: 0x2805e6cb0>, name:iPhone, manufac...",%,2020-09-21 10:32:19+03:00,2020-09-21 10:22:17+03:00,2020-09-21 10:22:26+03:00,0.225
3,HKQuantityTypeIdentifierWalkingDoubleSupportPe...,14.0,"<<HKDevice: 0x2805e6cb0>, name:iPhone, manufac...",%,2020-09-21 13:01:57+03:00,2020-09-21 12:50:43+03:00,2020-09-21 12:50:47+03:00,0.306
4,HKQuantityTypeIdentifierWalkingDoubleSupportPe...,14.0,"<<HKDevice: 0x2805e6cb0>, name:iPhone, manufac...",%,2020-09-21 13:01:57+03:00,2020-09-21 12:51:06+03:00,2020-09-21 12:52:19+03:00,0.275


In [60]:
my_walking_steady = my_all_data[my_all_data["type"] == "HKQuantityTypeIdentifierAppleWalkingSteadiness"].reset_index().drop(columns = ["index"])
my_walking_steady.head()

,type,sourceVersion,device,unit,creationDate,startDate,endDate,value
0,HKQuantityTypeIdentifierAppleWalkingSteadiness,2775.0.6,"<<HKDevice: 0x280540190>, name:iPhone, manufac...",%,2022-09-16 22:58:11+03:00,2022-06-25 03:00:00+03:00,2022-07-02 03:00:00+03:00,1.000000
1,HKQuantityTypeIdentifierAppleWalkingSteadiness,2775.0.6,"<<HKDevice: 0x280540190>, name:iPhone, manufac...",%,2022-09-16 22:58:11+03:00,2022-07-02 03:00:00+03:00,2022-07-09 03:00:00+03:00,1.000000
2,HKQuantityTypeIdentifierAppleWalkingSteadiness,2775.0.6,"<<HKDevice: 0x280540190>, name:iPhone, manufac...",%,2022-09-16 22:58:11+03:00,2022-07-10 03:00:00+03:00,2022-07-17 03:00:00+03:00,1.000000
3,HKQuantityTypeIdentifierAppleWalkingSteadiness,2775.0.6,"<<HKDevice: 0x280540190>, name:iPhone, manufac...",%,2022-09-16 22:58:11+03:00,2022-07-17 03:00:00+03:00,2022-07-24 03:00:00+03:00,0.945581
4,HKQuantityTypeIdentifierAppleWalkingSteadiness,2775.0.6,"<<HKDevice: 0x280540190>, name:iPhone, manufac...",%,2022-09-16 22:58:11+03:00,2022-07-24 03:00:00+03:00,2022-07-31 03:00:00+03:00,0.805529


Dataframes related to energy statistics

In [61]:
my_active_energy = my_all_data[my_all_data["type"] == "HKQuantityTypeIdentifierActiveEnergyBurned"].reset_index().drop(columns = ['index'])
my_active_energy.head()

,type,sourceVersion,device,unit,creationDate,startDate,endDate,value
0,HKQuantityTypeIdentifierActiveEnergyBurned,16.0,"<<HKDevice: 0x2806f0eb0>, name:iPhone, manufac...",kcal,2022-09-16 22:19:16+03:00,2022-09-16 00:01:22+03:00,2022-09-16 00:02:04+03:00,0.078
1,HKQuantityTypeIdentifierActiveEnergyBurned,16.0,"<<HKDevice: 0x2806f0eb0>, name:iPhone, manufac...",kcal,2022-09-16 22:19:16+03:00,2022-09-16 00:15:33+03:00,2022-09-16 00:15:44+03:00,0.038
2,HKQuantityTypeIdentifierActiveEnergyBurned,16.0,"<<HKDevice: 0x2806f0eb0>, name:iPhone, manufac...",kcal,2022-09-16 22:19:16+03:00,2022-09-16 09:31:12+03:00,2022-09-16 09:39:46+03:00,0.098
3,HKQuantityTypeIdentifierActiveEnergyBurned,16.0,"<<HKDevice: 0x2806f0eb0>, name:iPhone, manufac...",kcal,2022-09-16 22:19:16+03:00,2022-09-16 10:37:43+03:00,2022-09-16 10:44:24+03:00,0.638
4,HKQuantityTypeIdentifierActiveEnergyBurned,16.0,"<<HKDevice: 0x2806f0eb0>, name:iPhone, manufac...",kcal,2022-09-16 22:19:16+03:00,2022-09-16 10:55:21+03:00,2022-09-16 10:58:32+03:00,0.686


In [62]:
my_basal_energy = my_all_data[my_all_data["type"] == "HKQuantityTypeIdentifierBasalEnergyBurned"].reset_index().drop(columns = ['index'])
my_basal_energy.head()

,type,sourceVersion,device,unit,creationDate,startDate,endDate,value
0,HKQuantityTypeIdentifierBasalEnergyBurned,16.0,"<<HKDevice: 0x2806c0190>, name:iPhone, manufac...",kcal,2022-09-16 22:19:16+03:00,2022-09-15 22:19:01+03:00,2022-09-15 22:34:05+03:00,17.963
1,HKQuantityTypeIdentifierBasalEnergyBurned,16.0,"<<HKDevice: 0x2806c0190>, name:iPhone, manufac...",kcal,2022-09-16 22:19:16+03:00,2022-09-15 22:34:05+03:00,2022-09-15 22:49:09+03:00,17.965
2,HKQuantityTypeIdentifierBasalEnergyBurned,16.0,"<<HKDevice: 0x2806c0190>, name:iPhone, manufac...",kcal,2022-09-16 22:19:16+03:00,2022-09-15 22:49:09+03:00,2022-09-15 23:04:13+03:00,17.954
3,HKQuantityTypeIdentifierBasalEnergyBurned,16.0,"<<HKDevice: 0x2806c0190>, name:iPhone, manufac...",kcal,2022-09-16 22:19:16+03:00,2022-09-15 23:04:13+03:00,2022-09-15 23:18:47+03:00,17.351
4,HKQuantityTypeIdentifierBasalEnergyBurned,16.0,"<<HKDevice: 0x2806c0190>, name:iPhone, manufac...",kcal,2022-09-16 22:19:16+03:00,2022-09-15 23:18:47+03:00,2022-09-15 23:33:41+03:00,17.758


Dataframe related to headphone audio statistics

In [63]:
my_headphone_audio = my_all_data[my_all_data["type"] == "HKQuantityTypeIdentifierHeadphoneAudioExposure"].reset_index().drop(columns = ['index'])
my_headphone_audio.head()

,type,sourceVersion,device,unit,creationDate,startDate,endDate,value
0,HKQuantityTypeIdentifierHeadphoneAudioExposure,13.1.2,"<<HKDevice: 0x28065ead0>, name:EarPods, manufa...",dBASPL,2019-10-01 23:39:10+03:00,2019-10-01 23:29:23+03:00,2019-10-01 23:38:40+03:00,82.5818
1,HKQuantityTypeIdentifierHeadphoneAudioExposure,13.1.2,"<<HKDevice: 0x28065ead0>, name:EarPods, manufa...",dBASPL,2019-10-01 23:53:20+03:00,2019-10-01 23:42:19+03:00,2019-10-01 23:51:47+03:00,83.0852
2,HKQuantityTypeIdentifierHeadphoneAudioExposure,13.1.2,"<<HKDevice: 0x28065ead0>, name:EarPods, manufa...",dBASPL,2019-10-01 23:53:20+03:00,2019-10-01 23:51:48+03:00,2019-10-01 23:51:51+03:00,84.4894
3,HKQuantityTypeIdentifierHeadphoneAudioExposure,13.1.2,"<<HKDevice: 0x28065ead0>, name:EarPods, manufa...",dBASPL,2019-10-01 23:53:20+03:00,2019-10-01 23:51:50+03:00,2019-10-01 23:52:24+03:00,83.5434
4,HKQuantityTypeIdentifierHeadphoneAudioExposure,13.1.2,"<<HKDevice: 0x28065ead0>, name:EarPods, manufa...",dBASPL,2019-10-01 23:59:44+03:00,2019-10-01 23:52:46+03:00,2019-10-01 23:56:38+03:00,76.4813


### Exploratory data analysis of the dataframes

The dataframe containing all the data

In [64]:
print(my_all_data.shape, "\n")
print(my_all_data.dtypes)
my_all_data.describe()

(117983, 8) 

type                                object
sourceVersion                       object
device                              object
unit                                object
creationDate     datetime64[ns, UTC+03:00]
startDate        datetime64[ns, UTC+03:00]
endDate          datetime64[ns, UTC+03:00]
value                              float64
dtype: object


,value
count,117983.000000
mean,45.667557
std,102.935253
min,0.000000
25%,0.213000
50%,10.000000
75%,69.000000
max,11094.000000


In [65]:
print(my_steps.shape, "\n")
print(my_steps.dtypes)
my_steps.describe()

(24049, 8) 

type                                object
sourceVersion                       object
device                              object
unit                                object
creationDate     datetime64[ns, UTC+03:00]
startDate        datetime64[ns, UTC+03:00]
endDate          datetime64[ns, UTC+03:00]
value                              float64
dtype: object


,value
count,24049.000000
mean,121.505801
std,201.071968
min,1.000000
25%,13.000000
50%,34.000000
75%,154.000000
max,11094.000000


In [66]:
print(my_distances.shape, "\n")
print(my_distances.dtypes)
my_distances.describe()

(23992, 8) 

type                                object
sourceVersion                       object
device                              object
unit                                object
creationDate     datetime64[ns, UTC+03:00]
startDate        datetime64[ns, UTC+03:00]
endDate          datetime64[ns, UTC+03:00]
value                              float64
dtype: object


,value
count,23992.000000
mean,0.085136
std,0.146155
min,0.000390
25%,0.008880
50%,0.022840
75%,0.103450
max,7.895340


In [67]:
print(my_climbed.shape, "\n")
print(my_climbed.dtypes)
my_climbed.describe()

(4888, 8) 

type                                object
sourceVersion                       object
device                              object
unit                                object
creationDate     datetime64[ns, UTC+03:00]
startDate        datetime64[ns, UTC+03:00]
endDate          datetime64[ns, UTC+03:00]
value                              float64
dtype: object


,value
count,4888.000000
mean,1.939444
std,1.443292
min,1.000000
25%,1.000000
50%,1.000000
75%,3.000000
max,18.000000


In [68]:
print(my_step_length.shape, "\n")
print(my_step_length.dtypes)
my_step_length.describe()

(6433, 8) 

type                                object
sourceVersion                       object
device                              object
unit                                object
creationDate     datetime64[ns, UTC+03:00]
startDate        datetime64[ns, UTC+03:00]
endDate          datetime64[ns, UTC+03:00]
value                              float64
dtype: object


,value
count,6433.000000
mean,74.494482
std,14.136979
min,25.000000
25%,64.000000
50%,74.000000
75%,85.000000
max,136.000000


In [69]:
print(my_walking_speed.shape, "\n")
print(my_walking_speed.dtypes)
my_walking_speed.describe()

(6433, 8) 

type                                object
sourceVersion                       object
device                              object
unit                                object
creationDate     datetime64[ns, UTC+03:00]
startDate        datetime64[ns, UTC+03:00]
endDate          datetime64[ns, UTC+03:00]
value                              float64
dtype: object


,value
count,6433.000000
mean,4.838903
std,1.376157
min,1.548000
25%,3.708000
50%,4.788000
75%,6.048000
max,8.136000


In [70]:
print(my_walking_asymmetry.shape, "\n")
print(my_walking_asymmetry.dtypes)
my_walking_asymmetry.describe()

(2836, 8) 

type                                object
sourceVersion                       object
device                              object
unit                                object
creationDate     datetime64[ns, UTC+03:00]
startDate        datetime64[ns, UTC+03:00]
endDate          datetime64[ns, UTC+03:00]
value                              float64
dtype: object


,value
count,2836.000000
mean,0.070339
std,0.179761
min,0.000000
25%,0.000000
50%,0.000000
75%,0.040000
max,1.000000


In [71]:
print(my_walking_double_support.shape, "\n")
print(my_walking_double_support.dtypes)
my_walking_double_support.describe()

(4528, 8) 

type                                object
sourceVersion                       object
device                              object
unit                                object
creationDate     datetime64[ns, UTC+03:00]
startDate        datetime64[ns, UTC+03:00]
endDate          datetime64[ns, UTC+03:00]
value                              float64
dtype: object


,value
count,4528.000000
mean,0.269955
std,0.033310
min,0.172000
25%,0.241000
50%,0.268000
75%,0.298000
max,0.364000


In [72]:
print(my_walking_steady.shape, "\n")
print(my_walking_steady.dtypes)
my_walking_steady.describe()

(53, 8) 

type                                object
sourceVersion                       object
device                              object
unit                                object
creationDate     datetime64[ns, UTC+03:00]
startDate        datetime64[ns, UTC+03:00]
endDate          datetime64[ns, UTC+03:00]
value                              float64
dtype: object


,value
count,53.000000
mean,0.995304
std,0.027600
min,0.805529
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


Dataframes related to energy statistics

In [73]:
print(my_active_energy.shape, "\n")
print(my_active_energy.dtypes)
my_active_energy.describe()

(7140, 8) 

type                                object
sourceVersion                       object
device                              object
unit                                object
creationDate     datetime64[ns, UTC+03:00]
startDate        datetime64[ns, UTC+03:00]
endDate          datetime64[ns, UTC+03:00]
value                              float64
dtype: object


,value
count,7140.000000
mean,3.107266
std,7.728980
min,0.001000
25%,0.035000
50%,0.080000
75%,1.298750
max,87.983000


In [74]:
print(my_basal_energy.shape, "\n")
print(my_basal_energy.dtypes)
my_basal_energy.describe()

(19610, 8) 

type                                object
sourceVersion                       object
device                              object
unit                                object
creationDate     datetime64[ns, UTC+03:00]
startDate        datetime64[ns, UTC+03:00]
endDate          datetime64[ns, UTC+03:00]
value                              float64
dtype: object


,value
count,19610.000000
mean,38.663243
std,30.811339
min,0.203000
25%,11.454250
50%,26.253000
75%,74.805000
max,147.352000


Dataframe related to headphone audio statistics

In [75]:
print(my_headphone_audio.shape, "\n")
print(my_headphone_audio.dtypes)
my_headphone_audio.describe()

(18021, 8) 

type                                object
sourceVersion                       object
device                              object
unit                                object
creationDate     datetime64[ns, UTC+03:00]
startDate        datetime64[ns, UTC+03:00]
endDate          datetime64[ns, UTC+03:00]
value                              float64
dtype: object


,value
count,18021.000000
mean,64.490399
std,21.451476
min,0.000000
25%,55.232000
50%,67.173400
75%,80.299100
max,102.826000


### Visualizations of the categorical dataframes

Dataframes directly related to physical activity